## 1. Import Library and Dataset

In [1]:
import nltk # Imports the library
# nltk.download_shell() #Download the necessary datasets
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
%matplotlib inline

In [2]:
df = pd.read_csv('Dataset/Indonesian Sentiment Twitter Dataset Labeled.csv', sep = '\t')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10806 entries, 0 to 10805
Data columns (total 2 columns):
sentimen    10806 non-null int64
Tweet       10806 non-null object
dtypes: int64(1), object(1)
memory usage: 168.9+ KB


In [3]:
df.head()

,sentimen,Tweet
0,-1,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...
1,-1,kita lanjutkan saja diam ini hingga kau dan ak...
2,1,doa rezeki tak putus inna haa zaa larizquna ma...
3,1,makasih loh ntar kita bagi hasil aku 99 9 sisa...
4,-1,aku tak faham betul jenis orang malaysia yang ...


In [4]:
df.sentimen.value_counts()

 0    5327
-1    2887
 1    2592
Name: sentimen, dtype: int64

## 2. Import Clean-up Dataset

In [5]:
kamusalay_2 = pd.read_csv('Dataset/colloquial-indonesian-lexicon.csv')

kamusalay_2['Alay'] =kamusalay_2['slang']

kamusalay_2['KBBI'] =kamusalay_2['formal']
kamusalay_2.drop('slang formal In-dictionary context category1 category2 category3'.split(), axis = 1, inplace = True)
kamusalay_2.shape

(15006, 2)

In [6]:
kamusalay = pd.read_csv('Dataset/kamusalay.csv', header = None)
kamusalay.columns = ['Alay', 'KBBI']
kamusalay.shape

(238, 2)

In [7]:
kamus_alay = pd.merge(kamusalay, kamusalay_2, how = 'outer')
kamus_alay.drop_duplicates(subset = 'Alay', inplace = True)
kamus_alay.reset_index(inplace = True, drop = True)
# kamus_alay.shape

kamus = {}
# kamus_alay[kamus_alay['Alay'] == 'gw']
for index, row in kamus_alay.iterrows():
    kamus[row['Alay']] = row['KBBI']

In [8]:
kamus_alay['KBBI']

0                  saja
1                  saja
2                   aku
3               ah masa
4                  kamu
5                   apa
6                   apa
7                   apa
8                  saya
9              sabar ya
10             sabar ya
11             sabar ya
12             sabar ya
13      selamat tinggal
14                besok
15               banget
16      selamat tinggal
17                 bisa
18                 bisa
19             sebentar
20                 ayah
21                 ayah
22                 ayah
23                 ayah
24                 ayah
25                 bisa
26                 bisa
27                besok
28                besok
29                bosan
             ...       
4446    ditengah-tengah
4447           semuanya
4448           semuanya
4449          amin-amin
4450              habis
4451         kebanyakan
4452         melahirkan
4453               aduh
4454              cakep
4455                loh
4456            

In [9]:
kamusalay[kamusalay['Alay']== 'saya']

,Alay,KBBI


In [10]:
kamus_alay.to_csv('kamus_slang.csv')

## 3. Clean-up Dataset

### 3a. Clean up uppercase and hyperlink

In [8]:
def clean_link(text):
    #lowercase all
    text = text.lower()
    #hapus RT
    text=re.sub('^rt @[^\s]+','',text)
    #text = re.sub(',','',text)
    text=re.sub(r'http\S+','',text)
    #hapus \n
    text=re.sub('\n',' ',text)

    return text


In [9]:
df['clean_link'] = df['Tweet'].apply(clean_link)

In [10]:
df.to_csv('sentiment_analysis_ID_clean.csv')

### 3b. Hapus tanda baca dan angka

In [68]:
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def hapus_punc(text):
    hapus = string.punctuation + '0123456789'
    text = [char for char in text if char not in hapus]
    return ''.join(text)
    

In [12]:
df['hapus_punc'] = df['clean_link'].apply(hapus_punc)

In [13]:
df.to_csv('sentiment_analysis_ID_clean.csv')

### 3c. Substitusi slang words

In [14]:
def hapus_hurufdouble(s): 
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

def substitute_slang(text):
    text = text.split()
    text = [hapus_hurufdouble(word) for word in text]
    global kamus
    for i in range(len(text)):
        if text[i] in kamus.keys():
            text[i] = kamus[text[i]]
    
    return ' '.join(text)


In [15]:
df['substitute_slang'] = df['hapus_punc'].apply(substitute_slang)

In [16]:
df.to_csv('sentiment_analysis_ID_clean.csv')

### 3d. Stemming

In [17]:
def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
    return stemmer.stem(text)

In [18]:
df['stemming'] = df['substitute_slang'].apply(stemming)

In [19]:
df.to_csv('sentiment_analysis_ID_clean.csv')

### 3e. Buang stop words

In [20]:
def eliminate_stop(text):
    return ' '.join([word for word in text.split() if word not in stopwords.words('indonesian')])

In [21]:
df['eliminate_stop'] = df['stemming'].apply(eliminate_stop)

In [22]:
df.to_csv('sentiment_analysis_ID_clean.csv')

### 3f. Buang Noise (threshold 10)

In [23]:
# determine threshold
threshold = 10

In [24]:
daftar_kolom = list(df.columns)
daftar_kolom.remove('sentimen')
daftar_kolom

['Tweet',
 'clean_link',
 'hapus_punc',
 'substitute_slang',
 'stemming',
 'eliminate_stop']

In [25]:
word_tally = {}

for i in daftar_kolom:
    kotak = []
    for review in df[i].dropna():
        for a in review.split():
            kotak.append(a)
    tally = {}
    for j in kotak:
        if j in tally.keys():
            tally[j] += 1
        else : 
            tally[j] = 1
    word_tally[i] = tally

In [26]:
list_noise = pd.Series(word_tally['eliminate_stop'])
list_noise = list(list_noise[list_noise < threshold].index)

In [27]:
def eliminate_noise(text):
    global list_noise
    text = text.split()
    text = [word for word in text if word not in list_noise]
    return ' '.join(text)

In [28]:
df['eliminate_noise'] = df['eliminate_stop'].dropna().apply(eliminate_noise)

In [30]:
df.to_csv('sentiment_analysis_ID_clean.csv')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10806 entries, 0 to 10805
Data columns (total 8 columns):
sentimen            10806 non-null int64
Tweet               10806 non-null object
clean_link          10806 non-null object
hapus_punc          10806 non-null object
substitute_slang    10806 non-null object
stemming            10806 non-null object
eliminate_stop      10806 non-null object
eliminate_noise     10806 non-null object
dtypes: int64(1), object(7)
memory usage: 675.5+ KB


In [33]:
df[df['eliminate_stop'] == '']

,sentimen,Tweet,clean_link,hapus_punc,substitute_slang,stemming,eliminate_stop,eliminate_noise
30,0,aku tambah lagi ia,aku tambah lagi ia,aku tambah lagi ia,aku tambah lagi ia,aku tambah lagi ia,,
74,0,ini tiga aku satu aja gapunya ',ini tiga aku satu aja gapunya ',ini tiga aku satu aja gapunya,ini tiga aku satu saja enggak punya,ini tiga aku satu saja enggak punya,,
87,1,jadilah seperti aku,jadilah seperti aku,jadilah seperti aku,jadilah seperti aku,jadi seperti aku,,
96,0,kamu juga kan,kamu juga kan,kamu juga kan,kamu juga kan,kamu juga kan,,
122,0,karena aku kepo 11 13 20,karena aku kepo 11 13 20,karena aku kepo,karena aku ingin tahu,karena aku ingin tahu,,
180,-1,kok gitu sama aku,kok gitu sama aku,kok gitu sama aku,kok begitu sama aku,kok begitu sama aku,,
191,0,sama aku kapan,sama aku kapan,sama aku kapan,sama aku kapan,sama aku kapan,,
294,0,kamu akan tanya karena aku pun juga mau tanya,kamu akan tanya karena aku pun juga mau tanya,kamu akan tanya karena aku pun juga mau tanya,kamu akan tanya karena aku pun juga mau tanya,kamu akan tanya karena aku pun juga mau tanya,,
410,1,aku mau,aku mau,aku mau,aku mau,aku mau,,
496,1,aku mauuuu,aku mauuuu,aku mauuuu,aku mau,aku mau,,


### 3g. Buang Noise (threshold 20)

,sentimen,Tweet,clean_link,hapus_punc,substitute_slang,stemming,eliminate_stop,eliminate_noise,eliminate_noise_20,eliminate_noise_10
0,-1,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ini huhuhuhuhuhuh...,lagu bosan apa yang aku save ini huhuhuhuhuhuh...,lagu bosan save huhuhuhuhuhuhuhuhuhuu,lagu bosan save,lagu bosan save,lagu bosan save
1,-1,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kamu dan a...,kita lanjut saja diam ini hingga kamu dan aku ...,diam erti libat hati,diam erti hati,diam erti hati,diam erti hati
2,1,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki putus inna haa zaa larizquna maa la...,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p
3,1,makasih loh ntar kita bagi hasil aku 99 9 sisa...,makasih loh ntar kita bagi hasil aku 99 9 sisa...,makasih loh ntar kita bagi hasil aku sisanya...,terima kasih loh entar kita bagi hasil aku sis...,terima kasih loh entar kita bagi hasil aku sis...,terima kasih loh entar hasil sisa,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil
4,-1,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,faham jenis orang malaysia ret anak sangkut ho...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...


In [47]:
# determine threshold
threshold = 10

In [48]:
daftar_kolom = list(df.columns)
daftar_kolom.remove('sentimen')
daftar_kolom

['Tweet',
 'clean_link',
 'hapus_punc',
 'substitute_slang',
 'stemming',
 'eliminate_stop',
 'eliminate_noise']

In [49]:
word_tally = {}

for i in daftar_kolom:
    kotak = []
    for review in df[i].dropna():
        for a in review.split():
            kotak.append(a)
    tally = {}
    for j in kotak:
        if j in tally.keys():
            tally[j] += 1
        else : 
            tally[j] = 1
    word_tally[i] = tally

In [51]:
list_noise_20 = pd.Series(word_tally['eliminate_stop'])
list_noise_20 = list(list_noise_20[list_noise_20 < threshold].index)

In [52]:
def eliminate_noise(text):
    global list_noise_20
    text = text.split()
    text = [word for word in text if word not in list_noise]
    return ' '.join(text)

In [53]:
df['eliminate_noise_20'] = df['eliminate_stop'].dropna().apply(eliminate_noise)

In [55]:
df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10806 entries, 0 to 10805
Data columns (total 9 columns):
sentimen              10806 non-null int64
Tweet                 10806 non-null object
clean_link            10806 non-null object
hapus_punc            10806 non-null object
substitute_slang      10806 non-null object
stemming              10806 non-null object
eliminate_stop        10806 non-null object
eliminate_noise       10806 non-null object
eliminate_noise_20    10806 non-null object
dtypes: int64(1), object(8)
memory usage: 844.2+ KB


In [56]:
df.to_csv('sentiment_analysis_ID_clean.csv')

In [57]:
df.head()

,sentimen,Tweet,clean_link,hapus_punc,substitute_slang,stemming,eliminate_stop,eliminate_noise,eliminate_noise_20
0,-1,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ini huhuhuhuhuhuh...,lagu bosan apa yang aku save ini huhuhuhuhuhuh...,lagu bosan save huhuhuhuhuhuhuhuhuhuu,lagu bosan save,lagu bosan save
1,-1,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kamu dan a...,kita lanjut saja diam ini hingga kamu dan aku ...,diam erti libat hati,diam erti hati,diam erti hati
2,1,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki putus inna haa zaa larizquna maa la...,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p
3,1,makasih loh ntar kita bagi hasil aku 99 9 sisa...,makasih loh ntar kita bagi hasil aku 99 9 sisa...,makasih loh ntar kita bagi hasil aku sisanya...,terima kasih loh entar kita bagi hasil aku sis...,terima kasih loh entar kita bagi hasil aku sis...,terima kasih loh entar hasil sisa,terima kasih loh entar hasil,terima kasih loh entar hasil
4,-1,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,faham jenis orang malaysia ret anak sangkut ho...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...


In [70]:
coba = pd.Series(word_tally['eliminate_stop'])
coba[coba.index == 'gue']

gue    1
dtype: int64

In [66]:
coba.shape

(15855,)

In [100]:
df[df['hapus_punc'].str.contains(r'')]['hapus_punc'].values

array(['aku aja capek sama diriku sendiri apalagi kamu maaf ya  ',
       'terus kata senior gw lagi deki kalo kamu capek move on mending kamu berdoa gini ya tuhan kalo saya gabisa move ',
       'ho oh wkwk kok bisa itu lho tingkahnyaaaa heran aku wkwk lumayan kan nyes ilang dikit capekmu kelar kerja haha ',
       'mau pulang tapi rumahku jauh mau bolak balik nti aku capek',
       'saat capek banget dan kurang tidur aku lebih memilih menyendiri makan sendiri menghubungi orang tersayang dll rasanya tenang ',
       'istirahat sebentar capek dance  jam belum lagi nanti sore harus ketemu klien ',
       'apanya amplaz aku jalan jalan doang menyusuri trotoar capek terus ngadem di amplaz wkwkwk',
       'klw bo dengan ku yg cepet kluar ya say biar cepet selesai aku suka yg mainy sebentr gk bikin capek ',
       'selalu berhasil jadi mood booster ketika lagi capek makasih ya kamuosial media kesayangan quu',
       'cek emak aing dan itu bener bgt menjadi pendendam malah bikin hati capek',

In [90]:
df[df['eliminate_noise'] == '']

,sentimen,Tweet,clean_link,hapus_punc,substitute_slang,stemming,eliminate_stop,eliminate_noise,eliminate_noise_20
20,0,kau dah tak streak dengan aku pun,kau dah tak streak dengan aku pun,kau dah tak streak dengan aku pun,kamu sudah tak streak dengan aku pun,kamu sudah tak streak dengan aku pun,streak,,
25,0,high,high,high,high,high,high,,
30,0,aku tambah lagi ia,aku tambah lagi ia,aku tambah lagi ia,aku tambah lagi ia,aku tambah lagi ia,,,
45,1,diremes di enyot ngedusel dusel di nenen kamu ...,diremes di enyot ngedusel dusel di nenen kamu ...,diremes di enyot ngedusel dusel di nenen kamu ...,diremes di enyot ngedusel dusel di nenen kamu ...,diremes di enyot ngedusel dusel di nenen kamu ...,diremes enyot ngedusel dusel nenen mantap,,
68,0,hyung kamu kanibal kan ' ',hyung kamu kanibal kan ' ',hyung kamu kanibal kan,hyung kamu kanibal kan,hyung kamu kanibal kan,hyung kanibal,,
74,0,ini tiga aku satu aja gapunya ',ini tiga aku satu aja gapunya ',ini tiga aku satu aja gapunya,ini tiga aku satu saja enggak punya,ini tiga aku satu saja enggak punya,,,
87,1,jadilah seperti aku,jadilah seperti aku,jadilah seperti aku,jadilah seperti aku,jadi seperti aku,,,
96,0,kamu juga kan,kamu juga kan,kamu juga kan,kamu juga kan,kamu juga kan,,,
112,-1,wasyuu,wasyuu,wasyuu,wasyuu,wasyuu,wasyuu,,
122,0,karena aku kepo 11 13 20,karena aku kepo 11 13 20,karena aku kepo,karena aku ingin tahu,karena aku ingin tahu,,,


In [101]:
list_noise_20

['huhuhuhuhuhuhuhuhuhuu',
 'libat',
 'inna',
 'zaa',
 'larizquna',
 'maa',
 'lahu',
 'fadesungguhnya',
 'sisa',
 'sangkut',
 'hose',
 'barusan',
 'truk',
 'injek',
 'kopling',
 'shopping',
 'mamat',
 'cimb',
 'masam',
 'kevin',
 'keven',
 'rebut',
 'gimbot',
 'nunjuk',
 'twins',
 'ipad',
 'masin',
 'sd',
 'block',
 'penantiancintaeps',
 'gatal',
 'celah',
 'kangkang',
 'kg',
 'bujang',
 'ser',
 'habanero',
 'ganda',
 'kristalatu',
 'conversation',
 'iyee',
 'know',
 'these',
 'zikir',
 'retweetambil',
 'insha',
 'alla',
 'commitmentetakat',
 'hulu',
 'hilir',
 'together',
 'put',
 'effort',
 'bazir',
 'aah',
 'ayom',
 'interaksi',
 'segitu',
 'followback',
 'streak',
 'nunggun',
 'full',
 'nabung',
 'dubu',
 'over',
 'cite',
 'high',
 'opening',
 'hungerbelt',
 'spazio',
 'laahh',
 'siiss',
 'tabung',
 'pengantin',
 'natural',
 'hahahahahahahahahahaha',
 'sifu',
 'property',
 'approach',
 'investment',
 'etcemua',
 'gebetan',
 'ludah',
 'pacarin',
 'netweetjen',
 'lecturer',
 'tampal',

In [109]:
daftarkata = pd.Series(word_tally['eliminate_stop'])

In [115]:
daftarkata[(daftarkata<20)&(daftarkata>10)]

geli             12
rela             15
mampus           15
her              16
ria              14
pedas            15
dont             13
auto             17
adu              12
send             11
undang           12
ana              14
sisi             13
ba               14
bapa             14
level            12
aa               15
rules            16
wish             12
idm              18
takpe            11
can              15
sekian           17
tido             11
haih             16
ambik            12
kaget            13
meni             12
cium             12
komen            17
                 ..
band             12
wujud            15
program          14
ratnaarumpaet    11
kustom           14
ri               14
desa             15
bbm              11
tanggap          11
motif            11
kustomfest       11
didik            11
ringan           13
teaser           12
ratna            13
akibat           18
pilpres          16
suporter         17
lapor            16


In [117]:
for i in range(2,20,2):
    list_noise_temp = pd.Series(word_tally['eliminate_stop'])
    list_noise_temp = list(list_noise_temp[list_noise_temp < i].index)
    
    def eliminate_noise(text):
        global list_noise_temp
        text = text.split()
        text = [word for word in text if word not in list_noise]
        return ' '.join(text)
    
    df['eliminate_noise_'+str(i)] = df['eliminate_stop'].dropna().apply(eliminate_noise)

## 3. Data Visualization

In [5]:
df = pd.read_csv('sentiment_analysis_ID_clean.csv', index_col = 0)

In [6]:
df.head()

,sentimen,Tweet,clean_link,hapus_punc,substitute_slang,stemming,eliminate_stop,eliminate_noise,eliminate_noise_20,eliminate_noise_10,eliminate_noise_2,eliminate_noise_4,eliminate_noise_6,eliminate_noise_8,eliminate_noise_12,eliminate_noise_14,eliminate_noise_16,eliminate_noise_18
0,-1,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,lagu bosan apa yang aku save ini huhuhuhuhuhuh...,lagu bosan apa yang aku save ini huhuhuhuhuhuh...,lagu bosan save huhuhuhuhuhuhuhuhuhuu,lagu bosan save,lagu bosan save,lagu bosan save,lagu bosan save,lagu bosan save,lagu bosan save,lagu bosan save,lagu bosan save,lagu bosan save,lagu bosan save,lagu bosan save
1,-1,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kau dan ak...,kita lanjutkan saja diam ini hingga kamu dan a...,kita lanjut saja diam ini hingga kamu dan aku ...,diam erti libat hati,diam erti hati,diam erti hati,diam erti hati,diam erti hati,diam erti hati,diam erti hati,diam erti hati,diam erti hati,diam erti hati,diam erti hati,diam erti hati
2,1,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki tak putus inna haa zaa larizquna ma...,doa rezeki putus inna haa zaa larizquna maa la...,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p,doa rezeki putus haa min nya p
3,1,makasih loh ntar kita bagi hasil aku 99 9 sisa...,makasih loh ntar kita bagi hasil aku 99 9 sisa...,makasih loh ntar kita bagi hasil aku sisanya...,terima kasih loh entar kita bagi hasil aku sis...,terima kasih loh entar kita bagi hasil aku sis...,terima kasih loh entar hasil sisa,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil,terima kasih loh entar hasil
4,-1,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,aku tak faham betul jenis orang malaysia yang ...,faham jenis orang malaysia ret anak sangkut ho...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...,faham jenis orang malaysia ret anak geli bodoh...


In [7]:
df['Tweet'][0]

'lagu bosan apa yang aku save ni huhuhuhuhuhuhuhuhuhuuuuuuuuuuuuuu'

In [13]:
contoh = df['Tweet'][0].split()
contoh
max(set(contoh), key=contoh.count)

TypeError: 'key' is an invalid keyword argument for print()

In [26]:
from statistics import mode

def max_words(text):
    try :
        text = text.split()
        hasil = mode(text)
        return hasil
    except :
        return None

In [27]:
df['tweet_mode'] = df['Tweet'].apply(max_words)
df['clean_mode'] = df['eliminate_stop'].apply(max_words)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10806 entries, 0 to 10805
Data columns (total 20 columns):
sentimen              10806 non-null int64
Tweet                 10806 non-null object
clean_link            10806 non-null object
hapus_punc            10806 non-null object
substitute_slang      10804 non-null object
stemming              10804 non-null object
eliminate_stop        10673 non-null object
eliminate_noise       10270 non-null object
eliminate_noise_20    10270 non-null object
eliminate_noise_10    10270 non-null object
eliminate_noise_2     10270 non-null object
eliminate_noise_4     10270 non-null object
eliminate_noise_6     10270 non-null object
eliminate_noise_8     10270 non-null object
eliminate_noise_12    10270 non-null object
eliminate_noise_14    10270 non-null object
eliminate_noise_16    10270 non-null object
eliminate_noise_18    10270 non-null object
tweet_mode            3376 non-null object
clean_mode            3084 non-null object
dtypes: int64(

In [29]:
df['tweet_mode'].value_counts().head(10)

aku      497
kita     137
kamu     107
yg        89
yang      84
dan       80
di        68
dia       56
kau       49
orang     38
Name: tweet_mode, dtype: int64

In [40]:
df['clean_mode'].value_counts().head(10)

orang     99
anak      72
ya        57
batik     56
teman     27
cinta     27
jaga      21
sayang    21
kak       21
hati      20
Name: clean_mode, dtype: int64

In [41]:
daftar_kolom = list(df.columns)
daftar_kolom.remove('sentimen')


word_tally = {}

for i in daftar_kolom:
    kotak = []
    for review in df[i].dropna():
        for a in review.split():
            kotak.append(a)
    tally = {}
    for j in kotak:
        if j in tally.keys():
            tally[j] += 1
        else : 
            tally[j] = 1
    word_tally[i] = tally

In [43]:
daftar = pd.Series(word_tally['eliminate_stop']).sort_values(ascending = False)

daftar

orang             1229
ya                1020
anak               950
kalo               399
suka               349
kak                346
je                 330
hati               294
lihat              293
nya                284
pakai              281
allah              278
sih                273
cinta              269
teman              258
makan              255
sayang             253
banget             249
hidup              237
salah              233
batik              217
kena               205
amp                195
kat                191
kerja              187
kayak              185
kali               183
bantu              180
beli               173
main               172
                  ... 
jarangg              1
aduduu               1
cinhapp              1
pdi                  1
asn                  1
tumbl                1
merasmikn            1
sunrise              1
selipe               1
haish                1
betu                 1
davil                1
sulsel     

In [49]:
from nltk.tag import CRFTagger
import pycrfsuite
print(pycrfsuite.__file__)

/Users/frhamadiansyah/anaconda3/lib/python3.7/site-packages/pycrfsuite/__init__.py


In [50]:
ct = CRFTagger()

NameError: name 'pycrfsuite' is not defined

In [55]:
tag = 'lagu_NN bosan_NN apa_WH yang_SC aku_VB save_NN'
twit = 'lagu bosan save huhuhuhuhuhuhuhuhuhuu'

In [56]:
tag = tag.split()
twit = twit.split()

In [58]:
re.findall('^[a-z]+',tag[0])

['lagu']

In [62]:
isi = []
for word in tag:
    check = re.findall('^[a-z]+',word)
    print (check)
    if check[0] in twit:
        print ('la')
        isi.append(word)
isi

['lagu']
la
['bosan']
la
['apa']
['yang']
['aku']
['save']
la


['lagu_NN', 'bosan_NN', 'save_NN']

In [73]:
twit2 = 'terima kasih loh entar hasil sisa'
tag2 = 'terima_CD kasih_NN loh_NN entar_NN kita_PRP bagi_IN hasil_NN aku_NN 99_CD 9_CD sisanya_RB buat_VB kamu_NN'

In [74]:
twit2 = twit2.split()
tag2 = tag2.split()

In [80]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

isi = []
for word in tag2:
    check = re.findall('^[a-z]+',word)
    tag = re.findall('[A-Z]+',word)
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
#     print(check)
    try:
        check = stemmer.stem(check[0])
        if check in twit2:
#         print ('la')
            hasil = check+'_'+tag[0]
            isi.append(hasil)
    except:
        pass
    
isi


['terima_CD', 'kasih_NN', 'loh_NN', 'entar_NN', 'hasil_NN', 'sisa_RB']